In [ ]:
from dolfin import *
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline

parameters["form_compiler"]["optimize"]     = True
parameters["form_compiler"]["cpp_optimize"] = True

# Problem data

Most of this is ignored for now. Also, initial values are / can be computed with a linear approximation of the problem.

In [ ]:
class ExactInPlaneDisplacements(Expression):     
    """ data for the exact solutions with f,g as given in p.14 """
    def eval(self, values, x):
        values[0] = 0.0
        values[1] = -x[0]*x[1]/4.0
    def value_shape(self):
        return (2,)

class ExactOutOfPlaneDisplacements(Expression):
    """ data for the exact solutions with f,g as given in p.14 """
    def eval(self, values, x):
        # values[0] is for Q.sub(0) and values[1:] for Q.sub(1) when interpolating
        values[0] = 0.5* x[0]**2 * np.sin(x[1])
        values[1] = x[0] * np.sin(x[1])
        values[2] = - 0.5* x[0]**2 * np.cos(x[1])
    def value_shape(self):
        return (3,)

In [ ]:
class InitialInPlaneDisplacements(Expression):     
    def eval(self, values, x):
        values[0] = 0.0
        values[1] = -x[1]/10.0
    def value_shape(self):
        return (2,)

class InitialOutOfPlaneDisplacements(Expression):
    def eval(self, values, x):
        values[0] = 0.5*x[0]**2*(1-x[0])**2*sin(4*DOLFIN_PI*x[1])
        values[1] = x[0]*(1-x[0])*(1-2*x[0])*sin(4*DOLFIN_PI*x[1])
        values[2] = 2*DOLFIN_PI*x[0]**2*(1-x[0])**2*cos(4*DOLFIN_PI*x[1])
    def value_shape(self):
        return (3,)

In [ ]:
class DirichletBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return abs(x[0]) < DOLFIN_EPS and on_boundary

# Discrete spaces

In-plane displacements (IPD) are discretized with $(P_2)^2$. Out-of-plane displacements with a mixed function space $P_2 \ast (P_1)^2$.

In [ ]:
msh = RectangleMesh(Point(0.0, -0.5), Point(1.0, 0.5), 20, 20)

V = VectorFunctionSpace(msh, "Lagrange", 1, dim=2)        # in plane displacements (IPD)
DE = FiniteElement("Lagrange", msh.ufl_cell(), 1)         # out of plane displacements (OPD)
GE = VectorElement("Lagrange", msh.ufl_cell(), 1, dim=2)  # Gradients of OPD
W = FunctionSpace(msh, DE*GE)                             # mixed space for OPD

In [ ]:
tau_vals = [1.0]         # time step sizes
gamma = 1.0

def eps(u):
    return grad(u) + grad(u).T

f = Constant(0.0) #Constant(1e-5)
#tau = Constant(tau_vals[-1])

# Boundary conditions

I guess this needs some thinking...

In [ ]:
bcV = DirichletBC(V, Expression(("0.0", "-x[1]/10"), element=V.ufl_element()), DirichletBoundary())
bcW = DirichletBC(W.sub(0), Constant(0.0), DirichletBoundary())
bcDW = DirichletBC(W.sub(1), Constant((0.0, 0.0)), DirichletBoundary())

# Initial solution / guess

(Maybe) solve a linear problem to obtain some initial `w_, dw_`.

In [ ]:
wdwk = Function(W)   # current solution for out of plane
wdw_ = Function(W)   # solution from previous step

# FIXME: solve linearised problem to compute these?
wdw_init = InitialOutOfPlaneDisplacements(degree=1)
wdwk.interpolate(wdw_init)
wdw_.interpolate(wdw_init)

# Nonlinear problem

In [ ]:
threshold_iter_newton = 5           # Newton should converge in at most these steps
max_attempts_nonlinear = 10         # Number of times we try to decrease tau before giving up with Newton
max_step_size = 10**5               # max value of tau_vals
e_newton = 1.0e-5                   # tolerance for Newton
e_stop = msh.hmin()*1e-8

def setup_newton(W, wdwk, wdw_, u_, bcs, tau, omega=1.0, mu=1e5):
    p, q = TestFunctions(W)
    wk, dwk = split(wdwk)    # displacement and gradient of current solution
    w_, dw_ = split(wdw_)    # displacement and gradient of previous solution
    F = (1+gamma**2*tau) * inner(grad(dwk), grad(q))*dx - inner(grad(dw_), grad(q))*dx \
        + 2*tau*inner(inner(dwk, dwk)*dwk + dot(eps(u_), (dwk + dw_)/2), q)*dx \
        + mu*tau*inner(dwk - grad(wk), q)*dx + mu*tau*inner(dwk - grad(wk), grad(p))*dx
        #+ tau*inner(dwk - grad(wk), dwk - grad(wk)) * p * dx  # FIXME: this just wrong...
        #+ tau*inner(f, p)*dx \
        
    
    dpq = TrialFunction(W)  # direction along which to compute the derivative of the functional
    J = derivative(F, wdwk, dpq)
    problem = NonlinearVariationalProblem(F, wdwk, bcs, J)
    solver = NonlinearVariationalSolver(problem)
    prm = solver.parameters
    prm['newton_solver']['absolute_tolerance'] = e_newton/10.0
    prm['newton_solver']['relative_tolerance'] = e_newton
    prm['newton_solver']['maximum_iterations'] = 20
    prm['newton_solver']['relaxation_parameter'] = omega
    return solver

In [ ]:
# Old stuff:

#class OutOfPlane(NonlinearProblem):
#    def __init__(self, a, L):
#        NonlinearProblem.__init__(self)
#        self.L = L
#        self.a = a
#    def F(self, b, x):
#        assemble(self.L, tensor=b)
#    def J(self, A, x):
#        assemble(self.a, tensor=A)

#problem = OutOfPlane(J, F)  # FIXME! what about bcs??
#solver = NewtonSolver()

#solver.parameters["linear_solver"] = "lu" #"mumps"
#solver.parameters["convergence_criterion"] = "incremental"
#solver.parameters["relative_tolerance"] = e_newton

# Linear problem

In [ ]:
def linear_problem(u, u_, dwk, tau_value):
    v, z = TrialFunction(V), TestFunction(V)
    g  = Constant((0, 0))
    tau = Constant(tau_value)
    a = (1+tau)*inner(eps(v), eps(z))*dx
    L = inner(eps(u_), eps(z))*dx - tau*inner(outer(dwk, dwk), eps(z))*dx + tau*inner(g,z)*dx
    solve(a == L, u, bcV)
    return u

# Main loop

Iteratively solve the nonlinear and the linear problem until convergence. At each step the former is run with decreasingly small $\tau_k$ until Newton converges in under `threshold_iter_newton` (e.g. 5) iterations.

In [ ]:
set_log_level(PROGRESS)
#file_ipd = File("in-plane-displacements.pvd")
#file_opd = File("out-of-plane-displacements.pvd")
file = XDMFFile("displacements.xdmf")
file.parameters['flush_output'] = True
#info(file.parameters, True)  # check the console

k = 0 # current iteration
tau_vals = [1.0]
curls = []

wdwk.interpolate(wdw_init)
wdw_.interpolate(wdw_init)
wk, dwk = split(wdwk)    # displacement and gradient of current solution
w_, dw_ = split(wdw_)    # displacement and gradient of previous solution

uk  = Function(V)    # current solution for in-plane
u_  = Function(V)    # solution from previous step

u_init = InitialInPlaneDisplacements(degree=1)
uk.interpolate(u_init)
u_.interpolate(u_init)

P = VectorFunctionSpace(msh, "Lagrange", 1, dim=3)
fax = FunctionAssigner(P.sub(0), V.sub(0))
fay = FunctionAssigner(P.sub(1), V.sub(1))
faz = FunctionAssigner(P.sub(2), W.sub(0))

disp = Function(P)
disp.rename("disp", "displacement")
fax.assign(disp.sub(0), uk.sub(0))
fay.assign(disp.sub(1), uk.sub(1))
faz.assign(disp.sub(2), wdwk.sub(0))
#file << (disp, float(k))
file.write(disp, float(k))

while True:
    curls.append(assemble(curl(dwk)*dx))
    print("Step %d: tau_vals = %.3f, curl = %e" % (k, tau_vals[-1], curls[-1]))
    # solve OOP until newton ends in less than threshold_iter_newton steps
    j = 0
    omega = 1.0  # relaxation parameter for Newton
    while j < max_attempts_nonlinear:
        print("    Running Newton with tau = %.3f, omega = %.2f... " % (tau_vals[-1], omega), end='')
        solver = setup_newton(W, wdwk, wdw_, u_, [bcW, bcDW], tau_vals[-1], omega)
        try:
            niter, converged = solver.solve()
            if converged:
                wdw_.vector()[:] = wdwk.vector()   # FIXME: it is wrong to update at each attempt!!!
                if niter < threshold_iter_newton:
                    print("DONE in %d iterations." % niter)
                    break
                else:
                    print("CONVERGED in %d iterations" % niter)
        except RuntimeError as e:
            print("DIVERGED after %d iterations." % niter)  # FIXME: technically we are not sure this was the exception
            omega *= 0.5
        tau_vals[-1] *= 0.7
        j += 1

    u_.vector()[:] = uk.vector()
    linear_problem(uk, u_, dwk, tau_vals[-1])
    
    k += 1
    
    #file_ipd << (uk, float(k))
    #file_opd << (wdwk.split()[0], float(k))
    fax.assign(disp.sub(0), uk.sub(0))
    fay.assign(disp.sub(1), uk.sub(1))
    faz.assign(disp.sub(2), wdwk.sub(0))
    #file << (disp, float(k))
    file.write(disp, float(k))

    test1 = assemble(inner(grad(dwk-dw_), grad(dwk-dw_))*dx)
    test2 = assemble(inner(eps(uk-u_), eps(uk-u_))*dx)
    print("Step done with update of norm %.4f" % (test1+test2))
    if test1+test2 <= tau_vals[-1]*e_stop*min(1.0, tau_vals[-1]):
        print("End")
        break
    
    tau_vals.append(min(2*tau_vals[-1], max_step_size))

In [ ]:
pl.plot(tau_vals)

In [ ]:
pl.figure(figsize=(7,7))
_ = plot(wk, title="OPD")
#_ = plot(dwk, title="$\nabla OPD$")

In [ ]:
orig = Function(W)
orig.interpolate(wdw_init)

diff = project(orig.sub(0) - wk)
print(norm(diff.vector(), 'l2'))
print(norm(diff.vector(), 'linf'))

In [ ]:
pl.figure(figsize=(22,8))
pl.subplot(1,3,1)
#_ = plot(orig, title="Init: %e - %e" % (orig.sub(0).vector().min(), orig.sub(0).vector().max()))
plot(orig.sub(0), title="Init")
pl.subplot(1,3,2)
#_ = plot(dwk, title="Sol: %e - %e" % (wk.vector().min(), wk.vector().max()))
plot(wk, title="Sol")
pl.subplot(1,3,3)
_ = plot(diff, title="Diff: %e - %e" % (diff.vector().min(), diff.vector().max()))